In [2]:
import torch

model_path = '../models/Git/SRCNN.pth'# Cambia esto por la ruta al archivo

# Cargar el modelo
model_data = torch.load(model_path, map_location="cpu")

# Inspeccionar el contenido
if isinstance(model_data, dict):
    print("El archivo contiene un state_dict. Llaves disponibles:")
    for key in model_data.keys():
        print(f" - {key}")
else:
    print("El archivo contiene un modelo completo o un objeto desconocido:")
    print(type(model_data))


El archivo contiene un state_dict. Llaves disponibles:
 - conv1.weight
 - conv1.bias
 - conv2.weight
 - conv2.bias
 - conv3.weight
 - conv3.bias


In [3]:
import torch.nn as nn

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)  # Primera capa
        self.layer2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)  # Segunda capa
        self.layer3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)  # Tercera capa
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x


In [7]:
import torch

# Definir el modelo SRCNN
srcnn = SRCNN()

# Ruta al archivo de pesos
model_path = '../models/Git/SRCNN.pth'  # Cambia esto por la ruta al archivo

# Cargar los pesos y ajustar los nombres de las claves
state_dict = torch.load(model_path, map_location="cpu")
adjusted_state_dict = {
    "layer1.weight": state_dict["conv1.weight"],
    "layer1.bias": state_dict["conv1.bias"],
    "layer2.weight": state_dict["conv2.weight"],
    "layer2.bias": state_dict["conv2.bias"],
    "layer3.weight": state_dict["conv3.weight"],
    "layer3.bias": state_dict["conv3.bias"],
}

# Cargar los pesos ajustados en el modelo
srcnn.load_state_dict(adjusted_state_dict)

# Mover al dispositivo y establecer en modo evaluación
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
srcnn.to(device)
srcnn.eval()

print("Modelo SRCNN cargado exitosamente con los pesos ajustados.")


RuntimeError: Error(s) in loading state_dict for SRCNN:
	size mismatch for layer1.weight: copying a param with shape torch.Size([64, 1, 9, 9]) from checkpoint, the shape in current model is torch.Size([64, 3, 9, 9]).
	size mismatch for layer3.weight: copying a param with shape torch.Size([1, 32, 5, 5]) from checkpoint, the shape in current model is torch.Size([3, 32, 5, 5]).
	size mismatch for layer3.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([3]).

In [ ]:
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)  # Cambiar entrada a 1 canal
        self.layer2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.layer3 = nn.Conv2d(32, 1, kernel_size=5, padding=2)  # Salida de 1 canal
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x


adjusted_state_dict = {}

# Ajustar conv1 (de 1 canal a 3 canales)
adjusted_state_dict["layer1.weight"] = state_dict["conv1.weight"].repeat(1, 3, 1, 1) / 3
adjusted_state_dict["layer1.bias"] = state_dict["conv1.bias"]

# Copiar sin cambios para conv2
adjusted_state_dict["layer2.weight"] = state_dict["conv2.weight"]
adjusted_state_dict["layer2.bias"] = state_dict["conv2.bias"]

# Ajustar conv3 (de 1 canal a 3 canales)
adjusted_state_dict["layer3.weight"] = state_dict["conv3.weight"].repeat(3, 1, 1, 1)
adjusted_state_dict["layer3.bias"] = state_dict["conv3.bias"].repeat(3)


In [8]:
import torch
import torch.nn as nn

# Definir la arquitectura SRCNN para imágenes RGB
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)  # Tres canales de entrada
        self.layer2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.layer3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)  # Tres canales de salida
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Cargar pesos ajustados
model_path = '../models/Git/SRCNN.pth'
state_dict = torch.load(model_path, map_location="cpu")

# Ajustar los pesos para imágenes RGB
adjusted_state_dict = {}
adjusted_state_dict["layer1.weight"] = state_dict["conv1.weight"].repeat(1, 3, 1, 1) / 3
adjusted_state_dict["layer1.bias"] = state_dict["conv1.bias"]
adjusted_state_dict["layer2.weight"] = state_dict["conv2.weight"]
adjusted_state_dict["layer2.bias"] = state_dict["conv2.bias"]
adjusted_state_dict["layer3.weight"] = state_dict["conv3.weight"].repeat(3, 1, 1, 1)
adjusted_state_dict["layer3.bias"] = state_dict["conv3.bias"].repeat(3)

# Crear el modelo y cargar los pesos ajustados
srcnn = SRCNN()
srcnn.load_state_dict(adjusted_state_dict)

# Configurar dispositivo y modo evaluación
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
srcnn.to(device)
srcnn.eval()

print("Modelo SRCNN cargado exitosamente con los pesos ajustados.")


Modelo SRCNN cargado exitosamente con los pesos ajustados.


In [1]:
import os
import numpy as np
import torch
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image


class SRCNN(torch.nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.layer1 = torch.nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.layer2 = torch.nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.layer3 = torch.nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x


def apply_srcnn(lr_image, model, device):
    """
    Aplica el modelo SRCNN a la imagen de baja resolución.
    Args:
        lr_image (numpy.ndarray): Imagen de baja resolución
        model (torch.nn.Module): Modelo SRCNN
        device (torch.device): Dispositivo para ejecutar el modelo
    Returns:
        numpy.ndarray: Imagen de alta resolución generada
    """
    # Si la imagen tiene un solo canal, expandir a 3 canales
    if len(lr_image.shape) == 2 or lr_image.shape[-1] == 1:
        lr_image = np.stack([lr_image] * 3, axis=-1)
    lr_tensor = ToTensor()(lr_image).unsqueeze(0).to(device)
    with torch.no_grad():
        sr_tensor = model(lr_tensor)
    sr_image = ToPILImage()(sr_tensor.squeeze(0).cpu())
    return np.array(sr_image)


def process_npz_dataset(dataset_path, output_dir, model, device):
    """
    Procesa todas las imágenes en un archivo .npz usando SRCNN y guarda los resultados.
    Args:
        dataset_path (str): Ruta al archivo .npz
        output_dir (str): Ruta a la carpeta donde se guardarán las imágenes generadas
        model (torch.nn.Module): Modelo SRCNN
        device (torch.device): Dispositivo para ejecutar el modelo
    """
    # Crear carpeta de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Cargar el archivo .npz
    data = np.load(dataset_path)
    lr_images = data['arr_0']  # Baja resolución
    hr_images = data['arr_1']  # Alta resolución (ground truth)

    for i in range(len(lr_images)):
        # Obtener las imágenes de baja y alta resolución
        lr_image = (lr_images[i].astype(np.float32) / 255.0)  # Normalizar a [0, 1]
        hr_image = (hr_images[i].astype(np.float32) / 255.0)

        # Asegurar que ambas imágenes sean RGB
        if len(lr_image.shape) == 2 or lr_image.shape[-1] == 1:
            lr_image = np.stack([lr_image] * 3, axis=-1)
        if len(hr_image.shape) == 2 or hr_image.shape[-1] == 1:
            hr_image = np.stack([hr_image] * 3, axis=-1)

        # Pasar la imagen de baja resolución por el modelo SRCNN
        sr_image = apply_srcnn(lr_image, model, device)

        # Guardar las imágenes generadas
        Image.fromarray((lr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"lr_{i+1}.png"))
        Image.fromarray((hr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"hr_{i+1}.png"))
        Image.fromarray((sr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"sr_{i+1}.png"))

        print(f"Procesada muestra {i+1} -> Imágenes guardadas en: {output_dir}")


if __name__ == "__main__":
    # Ruta al archivo .npz
    dataset_path = "../data/confocal_exper_paired_filt_validsetR_256.npz"
    # Ruta para guardar las imágenes generadas
    output_dir = "../results/srcnn_outputs"

    # Configurar el dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Definir el modelo SRCNN
    srcnn = SRCNN()

    # Ajustar los pesos (usa adjusted_state_dict previamente generado)
    adjusted_state_dict = {
        "layer1.weight": torch.load("../models/Git/SRCNN.pth")["conv1.weight"].repeat(1, 3, 1, 1) / 3,
        "layer1.bias": torch.load("../models/Git/SRCNN.pth")["conv1.bias"],
        "layer2.weight": torch.load("../models/Git/SRCNN.pth")["conv2.weight"],
        "layer2.bias": torch.load("../models/Git/SRCNN.pth")["conv2.bias"],
        "layer3.weight": torch.load("../models/Git/SRCNN.pth")["conv3.weight"].repeat(3, 1, 1, 1),
        "layer3.bias": torch.load("../models/Git/SRCNN.pth")["conv3.bias"].repeat(3),
    }
    srcnn.load_state_dict(adjusted_state_dict)

    # Mover al dispositivo y configurar en modo evaluación
    srcnn.to(device)
    srcnn.eval()

    # Procesar el dataset
    process_npz_dataset(dataset_path, output_dir, srcnn, device)


Procesada muestra 1 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 2 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 3 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 4 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 5 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 6 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 7 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 8 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 9 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 10 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 11 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 12 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 13 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 14 -> Imágenes guardadas en: ../results/srcnn_outputs
P

In [2]:
import numpy as np

# Ruta al archivo .npz
dataset_path = "../data/confocal_exper_paired_filt_validsetR_256.npz"

# Cargar el dataset y explorar sus contenidos
data = np.load(dataset_path)

# Obtener las claves y dimensiones de los datos
dataset_info = {key: data[key].shape for key in data.keys()}
dataset_info


{'arr_0': (57, 256, 256, 3), 'arr_1': (57, 256, 256, 3)}

In [1]:
import os
import numpy as np
import torch
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image


class SRCNN(torch.nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.layer1 = torch.nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.layer2 = torch.nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.layer3 = torch.nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x


def apply_srcnn(lr_image, model, device):
    """
    Aplica el modelo SRCNN a la imagen de baja resolución.
    Args:
        lr_image (numpy.ndarray): Imagen de baja resolución
        model (torch.nn.Module): Modelo SRCNN
        device (torch.device): Dispositivo para ejecutar el modelo
    Returns:
        numpy.ndarray: Imagen de alta resolución generada
    """
    lr_tensor = ToTensor()(lr_image).unsqueeze(0).to(device)
    with torch.no_grad():
        sr_tensor = model(lr_tensor)
    sr_image = ToPILImage()(sr_tensor.squeeze(0).cpu())
    return np.array(sr_image)


def process_npz_dataset(dataset_path, output_dir, model, device):
    """
    Procesa todas las imágenes en un archivo .npz usando SRCNN y guarda los resultados.
    Args:
        dataset_path (str): Ruta al archivo .npz
        output_dir (str): Ruta a la carpeta donde se guardarán las imágenes generadas
        model (torch.nn.Module): Modelo SRCNN
        device (torch.device): Dispositivo para ejecutar el modelo
    """
    # Crear carpeta de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Cargar el archivo .npz
    data = np.load(dataset_path)
    lr_images = data['arr_0']  # Baja resolución
    hr_images = data['arr_1']  # Alta resolución (ground truth)

    for i in range(len(lr_images)):
        # Obtener las imágenes de baja y alta resolución
        lr_image = (lr_images[i].astype(np.float32) / 255.0)  # Normalizar a [0, 1]
        hr_image = (hr_images[i].astype(np.float32) / 255.0)

        # Pasar la imagen de baja resolución por el modelo SRCNN
        sr_image = apply_srcnn(lr_image, model, device)

        # Guardar las imágenes generadas
        Image.fromarray((lr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"lr_{i+1}.png"))
        Image.fromarray((hr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"hr_{i+1}.png"))
        Image.fromarray((sr_image * 255).astype(np.uint8)).save(os.path.join(output_dir, f"sr_{i+1}.png"))

        print(f"Procesada muestra {i+1} -> Imágenes guardadas en: {output_dir}")


if __name__ == "__main__":
    # Ruta al archivo .npz
    dataset_path = "../data/confocal_exper_paired_filt_validsetR_256.npz"
    # Ruta para guardar las imágenes generadas
    output_dir = "../results/srcnn_outputs"

    # Configurar el dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Definir el modelo SRCNN
    srcnn = SRCNN()

    # Ajustar los pesos (usa adjusted_state_dict previamente generado)
    adjusted_state_dict = {
        "layer1.weight": torch.load("../models/Git/SRCNN.pth")["conv1.weight"].repeat(1, 3, 1, 1) / 3,
        "layer1.bias": torch.load("../models/Git/SRCNN.pth")["conv1.bias"],
        "layer2.weight": torch.load("../models/Git/SRCNN.pth")["conv2.weight"],
        "layer2.bias": torch.load("../models/Git/SRCNN.pth")["conv2.bias"],
        "layer3.weight": torch.load("../models/Git/SRCNN.pth")["conv3.weight"].repeat(3, 1, 1, 1),
        "layer3.bias": torch.load("../models/Git/SRCNN.pth")["conv3.bias"].repeat(3),
    }
    srcnn.load_state_dict(adjusted_state_dict)

    # Mover al dispositivo y configurar en modo evaluación
    srcnn.to(device)
    srcnn.eval()

    # Procesar el dataset
    process_npz_dataset(dataset_path, output_dir, srcnn, device)


Procesada muestra 1 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 2 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 3 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 4 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 5 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 6 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 7 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 8 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 9 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 10 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 11 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 12 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 13 -> Imágenes guardadas en: ../results/srcnn_outputs
Procesada muestra 14 -> Imágenes guardadas en: ../results/srcnn_outputs
P